# Joining Data in SQL


In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [18]:
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import Table
from sqlalchemy import Column
from sqlalchemy import Integer, String
from sqlalchemy import inspect
import pandas as pd
from pprint import pprint as pp

In [20]:
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 300)
pd.set_option('display.expand_frame_repr', True)

In [23]:
engine = create_engine('sqlite:///countries.sql')
meta = MetaData(schema="countries")
conn = engine.connect()


In [5]:
engine= create_engine("sqlite:///countries.sql")
engine

Engine(sqlite:///countries.sql)

In [ ]:
SELECT *
FROM left_table
INNER JOIN right_table
ON left_table.id = right_table.id;

# Select name fields (with alias) and region 
SELECT cities.name AS city, countries.name AS country, region
FROM cities
  INNER JOIN countries
    ON cities.country_code = countries.code;

In [ ]:
SELECT c1.name AS city, c2.name AS country
FROM cities AS c1
INNER JOIN countries AS c2
ON c1.country_code = c2.code;

#  Select fields with aliases
SELECT c.code AS country_code, name, year, inflation_rate
FROM countries AS c
  #  Join to economies (alias e)
  INNER JOIN economies AS e
    # Match on code
ON c.code = e.code

In [ ]:
# 6. Select fields
SELECT c.code, name, region, e.year, fertility_rate, unemployment_rate
  # 1. From countries (alias as c)
  FROM countries AS c
  # 2. Join to populations (as p)
  INNER JOIN populations AS p
    # 3. Match on country code
    ON c.code = p.country_code
  # 4. Join to economies (as e)
  INNER JOIN economies AS e
    # 5. Match on country code and year
    ON c.code = e.code AND p.year = e.year;

# INNER JOIN via USING

In [ ]:
# if column the same can use USING(id) instead of c.id= p.id
SELECT *
FROM countries
  INNER JOIN economies
    USING(code)
    
# 4. Select fields
SELECT c.name AS country, continent, l.name AS language, official
  # 1. From countries (alias as c)
  FROM countries AS c
  # 2. Join to languages (as l)
  INNER JOIN languages AS l
    # 3. Match using code
    USING (code)

# Self-ish joins, just in CASE


In [ ]:
# 5. Select fields with aliases
SELECT p1.country_code,
       p1.size AS size2010,
       p2.size AS size2015
# 1. From populations (alias as p1)
FROM populations AS p1
  # 2. Join to itself (alias as p2)
  INNER JOIN populations AS p2
    # 3. Match on country code
    ON p1.country_code = p2.country_code
        # 4. and year (with calculation)
        AND p1.year = p2.year - 5;
        

SELECT p1.country_code,
       p1.size AS size2010, 
       p2.size AS size2015,
       # 1. calculate growth_perc
       ((p2.size - p1.size)/p1.size * 100.0) AS growth_perc
# 2. From populations (alias as p1)
FROM populations AS p1
  # 3. Join to itself (alias as p2)
  INNER JOIN populations AS p2
    # 4. Match on country code
    ON p1.country_code = p2.country_code
        # 5. and year (with calculation)
        AND p1.year = p2.year - 5;

In [ ]:
SELECT name, continent, code, surface_area,
    # 1. First case
    CASE WHEN surface_area > 2000000 THEN 'large'
        # 2. Second case
        WHEN surface_area > 350000 THEN 'medium'
        # 3. Else clause + end
        ELSE 'small' END
        # 4. Alias name
        AS geosize_group
# 5. From table
FROM countries;

In [ ]:
SELECT name, continent, code, surface_area,
    CASE WHEN surface_area > 2000000
            THEN 'large'
       WHEN surface_area > 350000
            THEN 'medium'
       ELSE 'small' END
       AS geosize_group
INTO countries_plus
FROM countries;

In [ ]:
SELECT country_code, size,
  CASE WHEN size > 50000000
            THEN 'large'
       WHEN size > 1000000
            THEN 'medium'
       ELSE 'small' END
       AS popsize_group
INTO pop_plus       
FROM populations
WHERE year = 2015;

# 5. Select fields
SELECT name, continent, geosize_group, popsize_group
# 1. From countries_plus (alias as c)
FROM countries_plus As c
  # 2. Join to pop_plus (alias as p)
  INNER JOIN pop_plus AS p
    # 3. Match on country code
    ON c.code = p.country_code
# 4. Order the table    
ORDER BY geosize_group;

# LEFT and RIGHT JOINs


In [ ]:
# Select the city name (with alias), the country code,
# the country name (with alias), the region,
# and the city proper population
SELECT c1.name AS city, code, c2.name AS country,
       region, city_proper_pop
# From left table (with alias)
FROM cities AS c1
  # Join to right table (with alias)
  INNER JOIN countries AS c2
    #  on country code
    ON c1.country_code = c2.code
# Order by descending country code
ORDER BY code DESC;


SELECT c1.name AS city, code, c2.name AS country,
       region, city_proper_pop
FROM cities AS c1
  # 1. Join right table (with alias)
  LEFT JOIN countries AS c2
    # 2. Match on country code
    ON c1.country_code = c2.code
# 3. Order by descending country code
ORDER BY code DESC;

In [ ]:

# Select country name AS country, the country's local name,
#the language name AS language, and
#the percent of the language spoken in the country
SELECT c.name AS country, local_name, l.name AS language, percent

#From left table (alias as c)
FROM countries AS c
  # 2. Join to right table (alias as l)
  LEFT JOIN languages AS l
    # 3. Match on fields
    ON c.code = l.code
# 4. Order by descending country
ORDER BY country DESC;

In [ ]:
# 5. Select name, region, and gdp_percapita
SELECT name, region, gdp_percapita
# 1. From countries (alias as c)
FROM countries AS c
  # 2. Left join with economies (alias as e)
  LEFT JOIN economies AS e
    # 3. Match on code fields
    ON c.code = e.code
# 4. Focus on 2010
WHERE year = 2010;

# Select fields
SELECT region, avg(gdp_percapita) AS avg_gdp
# From countries (alias as c)
FROM countries AS c
  # Left join with economies (alias as e)
  LEFT JOIN economies AS e
    # Match on code fields
   USING(code)
# Focus on 2010
WHERE year = 2010
# Group by region
GROUP BY region;

# Select fields
SELECT region, avg(gdp_percapita) AS avg_gdp
# From countries (alias as c)
FROM countries AS c
  # Left join with economies (alias as e)
  LEFT JOIN economies AS e
    # Match on code fields
   USING(code)
# Focus on 2010
WHERE year = 2010
# Group by region
GROUP BY region;
# desc order
ORDER BY avg_gdp

In [ ]:
# RIGHT JOIN
SELECT cities.name AS city, urbanarea_pop, countries.name AS country,
       indep_year, languages.name AS language, percent
FROM languages 
  RIGHT JOIN countries
    ON languages.code = countries.code
  RIGHT JOIN cities
    ON countries.code = cities.country_code
ORDER BY city, language;

# FULL JOINs


In [ ]:
SELECT name AS country, code, region, basic_unit
# 3. From countries
FROM countries
  # 4. Join to currencies
  FULL JOIN currencies
    # 5. Match on code
    USING (code)
# 1. Where region is North America or null
WHERE region = 'North America' OR region IS NULL
# 2. Order by region
ORDER BY region;

SELECT countries.name, code, languages.name AS language
FROM languages
  #1. Join to countries
  INNER JOIN countries
    USING (code)
# 2. Where countries.name starts with V or is null
WHERE countries.name LIKE 'V%' OR countries.name IS NULL
ORDER BY countries.name;

In [ ]:
# 7. Select fields (with aliases)
SELECT c1.name AS country, region, l.name AS language,
       basic_unit, frac_unit
# 1. From countries (alias as c1)
FROM countries AS c1
  # 2. Join with languages (alias as l)
  FULL JOIN languages AS l
    # 3. Match on code
    USING (code)
  # 4. Join with currencies (alias as c2)
  FULL JOIN currencies AS c2
    # 5. Match on code
    USING (code)
# 6. Where region like Melanesia and Micronesia
WHERE region LIKE 'M%esia';

# CROSSing the rubicon

In [ ]:
# 4. Select fields
SELECT c.name AS city, l.name AS language
# 1. From cities (alias as c)
FROM cities AS c        
  # 2. Join to languages (alias as l)
  CROSS JOIN languages AS l
# 3. Where c.name like Hyderabad
WHERE c.name LIKE 'Hyder%';

In [ ]:
# Select fields
SELECT c.name AS country, region, p.life_expectancy AS life_exp
# From countries (alias as c)
FROM countries AS c
  # Join to populations (alias as p)
  LEFT JOIN populations AS p
    # Match on country code
    ON c.code = p.country_code
# Focus on 2010
WHERE year = 2010
# Order by life_exp
ORDER by life_exp
# Limit to 5 records
LIMIT 5

# State of the UNION


In [ ]:
#tables stacking above each other

# Select fields from 2010 table
SELECT *
  # From 2010 table
  FROM economies2010
	# Set theory clause
	SET UNION
# Select fields from 2015 table
SELECT *
  # From 2015 table
  FROM economies2015
# Order by code and year
ORDER BY code, year;

In [ ]:
# Select field
SELECT country_code
  # From cities
  FROM cities
	# Set theory clause
	UNION
# Select field
SELECT code
  # From currencies
  FROM currencies
# Order by country_code
ORDER BY country_code;

In [ ]:
# for duplicates use UNION all

# Select fields
SELECT code, year
  # From economies
  FROM economies
	# Set theory clause
	SET UNION ALL
# Select fields
SELECT country_code, year
  # From populations
  FROM populations
# Order by code, year
ORDER BY code, year;

# INTERSECTional data science

In [ ]:
# good fof finding the same values
# syntax
SELECT id
FROM left_one 
INTERSECT
SELECT id
FROM right_one

# Select fields
SELECT code, year
  # From economies
  FROM economies
	# Set theory clause
	SET INTERSECT
# Select fields
SELECT country_code, year
  # From populations
  FROM populations
# Order by code and year
ORDER BY code, year;

In [ ]:
# see which countries have the same city names
#Select fields
SELECT name
  # From countries
  FROM countries
	# Set theory clause
	SET INTERSECT
# Select fields
SELECT name
  # From cities
  FROM cities;

# EXCEPTional

In [ ]:
# this includes the city with missing country codes 
# Select field
SELECT name
  # From cities
  FROM cities
	# Set theory clause
	EXCEPT
# Select field
SELECT capital
  # From countries
  FROM countries
# Order by result
ORDER BY name;


# Determine the names of capital cities that are not listed in the cities table.
# Select field
SELECT capital
  # From countries
  FROM countries
	# Set theory clause
	SET EXCEPT
# Select field
SELECT name
  # From cities
  FROM cities
# Order by ascending capital
ORDER BY capital;

# Semi-joins and Anti-joins


In [ ]:
# Select distinct fields
SELECT DISTINCT(name)
  # From languages
  FROM languages
# Where in statement
WHERE code IN
  # Subquery
  (SELECT code
   FROM countries
   WHERE region='Middle East')
# Order by name
ORDER BY name;


In [ ]:
# 3. Select fields
SELECT code, name
  # 4. From Countries
  FROM countries
  # 5. Where continent is Oceania
  WHERE continent = 'Oceania'
  	# 1. And code not in
  	AND code NOT IN
  	# 2. Subquery
  	(SELECT code
  	 FROM currencies);

In [ ]:
# Ientify the country codes that are included in either economies or currencies but not in populations.

# Select the city name
SELECT name
  # Alias the table where city name resides
  FROM cities AS c1
  # Choose only records matching the result of multiple set theory clauses
  WHERE country_code IN
(
    #Select appropriate field from economies AS e
    SELECT e.code
    FROM economies AS e
    # Get all additional (unique) values of the field from currencies AS c2  
    UNION
    SELECT c2.code
    FROM currencies AS c2
    # Exclude those appearing in populations AS p
    EXCEPT
    SELECT p.country_code
    FROM populations AS p
);

# Subqueries inside WHERE and SELECT clauses


In [ ]:
# You'll now try to figure out which countries 
#had high average life expectancies (at the country level) in 2015.

#example
SELECT *
  FROM populations
WHERE life_expectancy > 1.15 * 100
  AND year = 2015;
    
# Select fields
SELECT *
  # From populations
  FROM populations
# Where life_expectancy is greater than
WHERE life_expectancy >
  # 1.15 * subquery
  1.15 * (SELECT AVG(life_expectancy)
   FROM populations
   WHERE year = 2015) AND
  year = 2015;

In [ ]:
# Use your knowledge of subqueries in WHERE to get the urban area population for only capital cities.
#  2. Select fields
SELECT name, country_code, urbanarea_pop
   #. From cities
  FROM cities
 # Where city name in the field of capital cities
WHERE name IN
   # Subquery
  (SELECT capital
   FROM countries)
ORDER BY urbanarea_pop DESC;

In [ ]:
SELECT countries.name AS country, COUNT(*) AS cities_num
  FROM cities
    INNER JOIN countries
    ON countries.code = cities.country_code
GROUP BY country
ORDER BY cities_num DESC, country
LIMIT 9


SELECT countries.name AS country,
  (SELECT COUNT(*)
   FROM cities
   WHERE countries.code = cities.country_code) AS cities_num
FROM countries
ORDER BY cities_num DESC, country
LIMIT 9;


# Subquery inside FROM clause


In [ ]:
SELECT DISTINCT table1.col, subquery.maxperc
FROM table1,
    (SELECT cont, MAX(per) AS max_per
    FROM STATES 
    GROUP BY contient) AS subqery
WHERE

SELECT local_name, subquery.lang_num
  FROM countries,
  	(SELECT code, COUNT(*) AS lang_num
  	 FROM languages
  	 GROUP BY code) AS subquery
  WHERE countries.code = subquery.code
ORDER BY lang_num DESC;

In [ ]:
# Select fields
SELECT name, continent, inflation_rate
  # From countries
  FROM countries
	# Join to economies
	INNER JOIN economies
	# Match on code
	ON countries.code = economies.code
  # Where year is 2015
  WHERE year = 2015
    # And inflation rate in subquery (alias as subquery)
    AND inflation_rate IN (
        SELECT MAX(inflation_rate) AS max_inf
        FROM (
             SELECT name, continent, inflation_rate
             FROM countries
             INNER JOIN economies
             ON countries.code = economies.code
             WHERE year = 2015) AS subquery
      # Group by continent
        GROUP BY continent);

In [ ]:
-- Select fields
SELECT code, inflation_rate, unemployment_rate
  -- From economies
  FROM economies
  -- Where year is 2015 and code is not in
  WHERE year = 2015 AND code NOT IN
  	-- Subquery
  	(SELECT code
  	 FROM countries
  	 WHERE (gov_form = 'Constitutional Monarchy' OR gov_form LIKE '%Republic%'))
-- Order by inflation rate
ORDER BY inflation_rate;

# FINAL CHALLENGE

In [ ]:
-- Select fields
SELECT DISTINCT name, total_investment, imports
  -- From table (with alias)
  FROM countries AS c
    -- Join with table (with alias)
    LEFT JOIN economies AS e
      -- Match on code
      ON (c.code = e.code
        -- and code in Subquery
        AND c.code IN (
          SELECT l.code
          FROM languages AS l
          WHERE official = 'true'
        ) )
  -- Where region and year are correct
  WHERE region = 'Central America' AND year = 2015
-- Order by field
ORDER BY name;

In [ ]:
# Let's ease up a bit and calculate the average fertility rate for each region in 2015.
-- Select fields
SELECT region, continent, avg(fertility_rate) AS avg_fert_rate
  -- From left table
  FROM countries AS c
    -- Join to right table
    INNER JOIN populations AS p
      -- Match on join condition
      ON c.code = p.country_code
  -- Where specific records matching some condition
  WHERE year = 2015
-- Group appropriately
GROUP BY region, continent
-- Order appropriately
ORDER BY avg_fert_rate;

In [ ]:
# You are now tasked with determining the top 10 capital cities in Europe and the Americas in terms of a calculated percentage
#using city_proper_pop and metroarea_pop in cities.

-- Select fields
SELECT name, country_code, city_proper_pop, metroarea_pop,  
      -- Calculate city_perc
      city_proper_pop / metroarea_pop * 100 AS city_perc
  -- From appropriate table
  FROM cities
  -- Where 
  WHERE name IN
    -- Subquery
    (SELECT capital
     FROM countries
     WHERE (continent = 'Europe'
        OR continent LIKE '%America'))
       AND metroarea_pop IS NOT NULL
-- Order appropriately
ORDER BY city_perc DESC
-- Limit amount
LIMIT 10;